In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset
from lib.ehr.inpatient_interface import Inpatients

In [3]:
import logging
logging.root.level = logging.DEBUG


In [4]:
# Assign the folder of the dataset to `DATA_FILE`.
import dask

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")

with U.modified_environ(DATA_DIR=DATA_DIR), dask.config.set(scheduler='processes'):
    m4inpatient_dataset = load_dataset('M4ICU')
   

DEBUG:root:Loading dataframe files
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/adm_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/dx_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/static_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/obs_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/int_input.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/int_proc.csv
DEBUG:root:[DONE] Loading dataframe files
DEBUG:root:Matching admission_id
DEBUG:root:[DONE] Matching admission_id
DEBUG:root:Time casting..
DEBUG:root:[DONE] Time casting..
DEBUG:absl:Constructing dx_icd9 (<class 'lib.ehr.coding_scheme.DxICD9'>) scheme
DEBUG:absl:Constructing dx_icd10 (<class 'lib.ehr.coding_scheme.DxICD10'>) scheme
DEBUG:absl:Constructing dx_icd9 (<class 'lib.ehr.coding_scheme.DxICD9'>) scheme
DEBUG:absl:Constructing dx_icd9 (<class 'lib.ehr.coding_

In [5]:
splits = m4inpatient_dataset.random_splits([0.1, 0.7], 42, 'subjects')



In [6]:
preprocessing = m4inpatient_dataset.fit_preprocessing(splits[0])

In [7]:

m4inpatient_dataset.apply_preprocessing(preprocessing)

DEBUG:root:Removed 2431596 (0.024) outliers from obs


In [8]:
# from concurrent.futures import ThreadPoolExecutor
# with dask.config.set(pool=ThreadPoolExecutor(12)):
with dask.config.set(scheduler='processes', num_workers=12):
    m4inaptients = Inpatients(m4inpatient_dataset, splits[0], num_workers=12)

DEBUG:root:Loading subjects..
/home/asem/GP/ICENODE/notebooks/mimic_icu/../../lib/ehr/dataset.py:956: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  dob = anchor_date + anchor_age
DEBUG:absl:Constructing mimic4_eth32 (<class 'lib.ehr.coding_scheme.MIMIC4Eth32'>) scheme
DEBUG:absl:Constructing mimic4_eth5 (<class 'lib.ehr.coding_scheme.MIMIC4Eth5'>) scheme
DEBUG:root:Extracting dx codes...
DEBUG:absl:Constructing dx_icd10 (<class 'lib.ehr.coding_scheme.DxICD10'>) scheme
DEBUG:absl:Constructing dx_icd9 (<class 'lib.ehr.coding_scheme.DxICD9'>) scheme
                            dx_icd10->dx_icd9 Unrecognised t_codes
                            (169):
                            ['041.41', '041.42', '041.43', '041.49', '173.00', '173.01', '173.02', '173.09', '173.10', '173.11', '173.12', '173.19', '173.20', '173.21', '173.22', '173.29', '173.30', '173.31', '173.32', '173.39']...
                            dx_icd10->dx_icd9 Unrecognised s_codes

In [9]:
m4inaptients.size_in_bytes() / 1024 ** 3

0.8040173463523388

In [10]:
m4inaptients_jax = m4inaptients.to_jax_arrays(splits[0])

DEBUG:jax._src.xla_bridge:Initializing backend 'cpu'
DEBUG:jax._src.xla_bridge:Backend 'cpu' initialized
DEBUG:jax._src.xla_bridge:Initializing backend 'cuda'
DEBUG:jax._src.xla_bridge:Backend 'cuda' initialized
DEBUG:jax._src.xla_bridge:Initializing backend 'rocm'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: Interpreter CUDA
DEBUG:jax._src.xla_bridge:Initializing backend 'tpu'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'


In [11]:
m4inaptients_jax.size_in_bytes() / 1024 ** 3

0.7881306745111942

In [12]:
len(m4inaptients_jax.subjects)

5083

In [13]:
m4inaptients_jax.n_admissions()

17720

In [14]:
m4inaptients.n_segments()

884001

In [15]:
m4inaptients.n_obs_times()

730030

In [16]:
m4inaptients.p_obs()

0.22610260308937077

In [17]:
# import numpy as np
# import matplotlib.pyplot as plt

# a = m4inaptients.obs_coocurrence_matrix
# a = np.array(a)
# plt.imshow(a, cmap='hot', interpolation='nearest')
# plt.show()

In [18]:
s = m4inaptients_jax.subjects[splits[0][0]]

In [19]:
s

Inpatient(
  subject_id=13397616,
  static_info=InpatientStaticInfo(
    gender='F',
    date_of_birth=Timestamp('2047-01-01 00:00:00'),
    ethnicity=bool[5],
    ethnicity_scheme=<lib.ehr.coding_scheme.MIMIC4Eth5 object at 0x7fc72179fe20>,
    constant_vec=bool[6]
  ),
  admissions=[
    InpatientAdmission(
      admission_id=26746906,
      admission_dates=(
        Timestamp('2138-06-25 17:03:00'),
        Timestamp('2138-06-29 12:19:00')
      ),
      dx_codes=CodesVector(
        vec=bool[17375],
        scheme=<lib.ehr.coding_scheme.DxICD9 object at 0x7fc72181f520>
      ),
      dx_codes_history=CodesVector(
        vec=bool[17375],
        scheme=<lib.ehr.coding_scheme.DxICD9 object at 0x7fc72181f520>
      ),
      outcome=CodesVector(
        vec=bool[2081],
        scheme=<lib.ehr.outcome.OutcomeExtractor object at 0x7fc794f99c40>
      ),
      observables=[
        InpatientObservables(time=f32[3], value=f16[3,60], mask=bool[3,60]),
        InpatientObservables(time=f32[

In [20]:
m4inaptients_jax.outcome_frequency_partitions(5, splits[0])

DEBUG:jax._src.interpreters.pxla:Compiling fn for with global shapes and types [ShapedArray(int32[], weak_type=True), ShapedArray(bool[2081])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling fn for with global shapes and types [ShapedArray(int32[], weak_type=True), ShapedArray(int32[2081], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling fn for with global shapes and types [ShapedArray(int32[2081], weak_type=True), ShapedArray(int32[2081], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 

DEBUG:jax._src.interpreters.pxla:Compiling _cumulative_reduction for with global shapes and types [ShapedArray(float32[2081])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling searchsorted for with global shapes and types [ShapedArray(float32[2081]), ShapedArray(float32[4])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _unstack for with global shapes and types [ShapedArray(int32[4])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling fn for with global shapes and types [ShapedArr

DEBUG:jax._src.interpreters.pxla:Compiling gather for with global shapes and types [ShapedArray(int32[2081]), ShapedArray(int32[1])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]


[Array([   0,    1,    2, ..., 1858, 2039, 1989], dtype=int32),
 Array([1575,  647, 1907, 1511,  119, 1478, 1606,  380, 1830, 1486, 1405,
         367,  374, 1861, 1402, 1946, 1811,  382, 1959, 1592, 1862, 1596,
        1885, 1950, 1504, 1466, 1395, 1392, 1398, 1412, 1417, 1422, 1607,
        1549, 1401, 1404, 1550, 1810, 1621], dtype=int32),
 Array([1616, 1489, 1562, 1516, 1620, 1615, 1408, 1958, 1828, 1791, 1894,
        1961, 1835, 1852, 1631], dtype=int32),
 Array([1482, 2061, 1962, 1386, 1407, 1387, 1954], dtype=int32),
 Array([1595, 1490, 1610, 1388], dtype=int32)]

In [21]:
s.admissions[0].observables[4].time

Array([], dtype=float32)

In [22]:
s.static_info

InpatientStaticInfo(
  gender='F',
  date_of_birth=Timestamp('2047-01-01 00:00:00'),
  ethnicity=bool[5],
  ethnicity_scheme=<lib.ehr.coding_scheme.MIMIC4Eth5 object at 0x7fc72179fe20>,
  constant_vec=bool[6]
)

In [23]:
s.static_info.age(s.admissions[0].admission_dates[0])

91.47707049965777

### التدريب على نموذج المعادلات التفاضلية الاعتيادية العصبية


In [24]:
from lib.ml.in_icenode import InICENODE, InICENODEDimensions
import jax.random as jrandom

DEBUG:matplotlib:matplotlib data path: /home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/asem/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CACHEDIR=/home/asem/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/asem/.cache/matplotlib/fontlist-v330.json
/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/equinox/_ad.py:753: UserWarning: As of Equinox 0.10.7, `equinox.filter_custom_vjp.defvjp` is deprecated in favour of `.def_fwd` and `.def_bwd`. This new API supports symbolic zeros, which allow for more efficient autodifferentiation rules. In particular:
- the fwd and bwd functions take an extra `perturbed` argument, which     indicates which primals actually need a gradient. You can use this     to skip computing the gradient for any unperturbed value. (You can     also safely just ignore this if you wish.)
- `None` was pre

In [25]:
dims = InICENODEDimensions(state_m=15, 
                state_dx_e=10,
                state_obs_e=25,
                input_e=10,
                proc_e=10,
                demo_e=5,
                int_e=15)
key = jrandom.PRNGKey(0)

m = InICENODE(dims=dims, 
              scheme=m4inpatient_dataset.scheme,
              key=key)

DEBUG:jax._src.interpreters.pxla:Compiling _threefry_seed for with global shapes and types [ShapedArray(int32[])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _threefry_split for with global shapes and types [ShapedArray(uint32[2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _unstack for with global shapes and types [ShapedArray(uint32[5,2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _threefry_split for with global shapes and types [ShapedArray(uint32[2])]. Argument mapping: (GSPMDSharding({r

DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling dynamic_slice for with global shapes and types [ShapedArray(uint32[2,2]), ShapedArray(int32[]), ShapedArray(int32[])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling squeeze for with global shapes and types [ShapedArray(uint32[1,2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _unstack for with global shapes and types [ShapedArray(uint32[2,2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_repl

DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _threefry_split for with global shapes and types [ShapedArray(uint32[2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _unstack for with global shapes and types [ShapedArray(uint32[1,2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_

DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float

DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform

DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform

DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform

DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling dynamic_slice for with global shapes and types [ShapedArray(uint32[4,2]), ShapedArray(int32[]), ShapedArray(int32[])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and ty

DEBUG:jax._src.interpreters.pxla:Compiling fn for with global shapes and types [ShapedArray(float32[], weak_type=True), ShapedArray(float32[250,65])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling fn for with global shapes and types [ShapedArray(float32[], weak_type=True), ShapedArray(float32[250])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling fn for with global shapes and types [ShapedArray(float32[], weak_type=True), ShapedArray(float32[250,250])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 de

DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[CpuDevice(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float

In [47]:
m.batch_predict(m4inaptients_jax, splits[0][:10])

DEBUG:jax._src.interpreters.pxla:Compiling embed_segment for with global shapes and types [ShapedArray(float32[25,7]), ShapedArray(float32[25]), ShapedArray(float32[5,25]), ShapedArray(float32[5]), ShapedArray(float32[50,197]), ShapedArray(float32[50]), ShapedArray(float32[10,50]), ShapedArray(float32[10]), ShapedArray(float32[50,51]), ShapedArray(float32[50]), ShapedArray(float32[10,50]), ShapedArray(float32[10]), ShapedArray(float32[75,25]), ShapedArray(float32[75]), ShapedArray(float32[15,75]), ShapedArray(float32[15]), ShapedArray(float32[197]), ShapedArray(bool[51]), ShapedArray(float16[7])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}),

obs.time: [5.95]
t0: 5.95
t1: 6.85


XlaRuntimeError: INTERNAL: Generated function failed: CpuCallback error: RuntimeError: Must have (t1 - t0) * dt0 >= 0, we instead got t1 with value Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=2/0)> and type <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>, t0 with value Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=2/0)> and type <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>, dt0 with value Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=2/0)> and type <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>

At:
  /home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/equinox/_errors.py(56): raises
  /home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/jax/_src/callback.py(186): _flat_callback
  /home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/jax/_src/callback.py(46): pure_callback_impl
  /home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/jax/_src/callback.py(108): _callback
  /home

In [29]:
m4inpatient_dataset.df['adm']['adm_interval'].min()

0.016666666666666666

In [ ]:
## Check sorted!
## NOOOOOO, remove subjects with at least one adm_interval of negative values.

In [54]:
m4inaptients_jax.subjects[splits[0][0]].admissions[0].interventions

InpatientInterventions(
  proc=None,
  input_=InpatientInput(
    index=i32[144],
    rate=f32[144],
    starttime=f32[144],
    endtime=f32[144],
    size=469
  ),
  time=[
    (0.0, 5.366666666666666),
    (5.366666666666666, 5.783333333333333),
    (5.783333333333333, 5.95),
    (5.95, 6.85),
    (6.85, 6.866666666666666),
    (6.866666666666666, 6.95),
    (6.95, 6.966666666666667),
    (6.966666666666667, 7.966666666666667),
    (7.966666666666667, 9.416666666666666),
    (9.416666666666666, 11.083333333333334),
    (11.083333333333334, 11.1),
    (11.1, 11.116666666666667),
    (11.116666666666667, 11.366666666666667),
    (11.366666666666667, 12.616666666666667),
    (12.616666666666667, 12.85),
    (12.85, 13.066666666666666),
    (13.066666666666666, 13.5),
    (13.5, 13.95),
    (13.95, 15.066666666666666),
    (15.066666666666666, 15.083333333333334),
    (15.083333333333334, 16.116666666666667),
    (16.116666666666667, 17.116666666666667),
    (17.116666666666667, 17.18333

In [46]:
import numpy as np
for i, s in m4inaptients.subjects.items():
    for a in s.admissions:
        time = a.interventions.time
        obs = a.observables
        if any(t1 - t0 < 1e-2 for (t0, t1) in time):
            print(i, time)
        assert len(time) == len(obs)
        if any((o.time < t0).any() or (o.time > t1).any() for ((t0, t1), o) in zip(time, obs)):
            print(i, o, time)
        
        if any((o.time[:-1] >= o.time[1:]).any() for o in obs):
            print(obs)

In [33]:
# Check proc/input start_time/end_time:
# Check obs times

In [50]:
a = np.array([1.5])
a.dtype

dtype('float64')

In [55]:
import jax.numpy as jnp

In [56]:
b = jnp.array(a, dtype=a.dtype)

/tmp/ipykernel_148726/4113290510.py:1: UserWarning: Explicitly requested dtype float64 requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  b = jnp.array(a, dtype=a.dtype)
